<a href="https://colab.research.google.com/github/R-A-P-I-D-Internship/Safety-Monitoring-on-a-Construction-Site/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import the dataset from roboflow using the api key

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY")
project = rf.workspace("YOUR_WORKSPACE_NAME").project("YOUR_PROJECT_NAME")
version = project.version(PROJECT_VERSION)
dataset = version.download("yolov5")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 507.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to fire-hazard-detection-2 in yolov5pytorch:: 100%|██████████| 10326/10326 [00:02<00:00, 3925.93it/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Image Augmentation - Conversion to greyscale

import os
import cv2
def loop_images(directory):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            images.append(os.path.join(directory, filename))
    return images
images = loop_images('PATH_TO_TEST_IMAGES')
for image in images:
    img = cv2.imread(image)
    if img is None:
        print(f"Error loading image {image}")
        continue
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    base_name = os.path.splitext(image)[0]
    cv2.imwrite(base_name + '_Converted_gray.jpg', gray)


In [ ]:
#Image Augmentation - Conversion to motion Blur

import os
import cv2
import numpy as np

def loop_images(directory):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") and "_Converted_gray" not in filename:
            images.append(os.path.join(directory, filename))
    return images

images = loop_images('PATH_TO_TEST_IMAGES')

kernel_size = 30

kernel_v = np.zeros((kernel_size, kernel_size))

kernel_v[:, int((kernel_size - 1)/2)] = np.ones(kernel_size)


kernel_v /= kernel_size

for image_path in images:
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error loading image {image_path}")
        continue
    vertical_mb = cv2.filter2D(img, -1, kernel_v)
    base_name = os.path.splitext(image_path)[0]
    cv2.imwrite(base_name + '_vertical_motion_blur.jpg', vertical_mb)

In [ ]:
#Image Augmentation - Adding noise to the images
import os
import random
import cv2

def add_noise(img):

    row, col = img.shape
    number_of_pixels = random.randint(300, 10000)
    for i in range(number_of_pixels):
        y_coord = random.randint(0, row - 1)
        x_coord = random.randint(0, col - 1)
        img[y_coord, x_coord] = 255
    number_of_pixels = random.randint(300, 10000)
    for i in range(number_of_pixels):
        y_coord = random.randint(0, row - 1)
        x_coord = random.randint(0, col - 1)
        img[y_coord, x_coord] = 0
    return img

def loop_images(directory):

    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") and "_Converted_gray" not in filename and "_vertical_motion_blur" not in filename:
            images.append(os.path.join(directory, filename))
    return images
images = loop_images('PATH_TO_TEST_IMAGES')

for image_path in images:
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error loading image {image_path}")
        continue
    if len(img.shape) == 3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray_img = img
    noisy_img = add_noise(gray_img)
    base_name = os.path.splitext(image_path)[0]
    cv2.imwrite(base_name + '_salt_and_pepper.jpg', noisy_img)

In [ ]:
import os
import cv2

def loop_images(directory):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") and "_Converted_gray" not in filename and "_vertical_motion_blur" not in filename and "_salt_and_pepper" not in filename:
            images.append(os.path.join(directory, filename))
    return images

def adjust_brightness_contrast(img, alpha, beta):
    return cv2.convertScaleAbs(img, alpha=alpha, beta=beta)


directory = 'PATH_TO_TEST_IMAGES'
images = loop_images(directory)

for image_path in images:

    img = cv2.imread(image_path)
    if img is None:
        print(f"Error loading image {image_path}")
        continue

    brightened_img = adjust_brightness_contrast(img, alpha=3, beta=50)
    darkened_img = adjust_brightness_contrast(img, alpha=0.4, beta=-5)
    base_name = os.path.splitext(image_path)[0]
    brightened_path = base_name + '_brightened.jpg'
    darkened_path = base_name + '_darkened.jpg'

    cv2.imwrite(brightened_path, brightened_img)
    cv2.imwrite(darkened_path, darkened_img)

In [ ]:
#Duplication of labels for the augmented Images
import os
import shutil

def duplicate_labels_for_augmentations(label_dir):

    original_labels = [f for f in os.listdir(label_dir) if f.endswith(".txt")]

    for original_label in original_labels:
        base_name = os.path.splitext(original_label)[0]
        augmented_image_types = ["_Converted_gray","_brightened.txt", "_darkened.txt", "_salt_and_pepper.txt", "_vertical_motion_blur.txt"]
        for aug_type in augmented_image_types:
            augmented_label_path = os.path.join(label_dir, base_name + aug_type)
            shutil.copyfile(os.path.join(label_dir, original_label), augmented_label_path)


label_dir = 'PATH_TO_THE_FOLDER_CONTAINING_THE_LABELS_FOR_THE_TESTING_IMAGES'

duplicate_labels_for_augmentations(label_dir)

In [ ]:
#Performing the train test and validation split

#PERFORMING TRAIN TEST AND VALIDATE SPLIT


from sklearn.model_selection import train_test_split
import os
import shutil
def list_images_and_labels(image_dir, label_dir):
    images = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]
    labels = [f.replace(".jpg", ".txt") for f in images]
    return images, labels
image_dir = 'PATH_TO_THE_FODLER_CONTAINING_THE_IMAGES'
label_dir = 'PATH_TO_THE_FODLER_CONTAINING_THE_LABELS_FOR_THE_IMAGES'

#Create three directories[Train,Validate and Test] each of which contain two subdirectories called label and images and put in their respective paths

train_image_dir = '/Train/images'
train_label_dir = '/Train/labels'
val_image_dir = '/Validate/images'
val_label_dir = '/Validate/labels'
test_image_dir = '/Test/images'
test_label_dir = '/Test/labels'

os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)
os.makedirs(test_image_dir, exist_ok=True)
os.makedirs(test_label_dir, exist_ok=True)
images, labels = list_images_and_labels(image_dir, label_dir)
x_main, x_test, y_main, y_test = train_test_split(images, labels, test_size=0.15, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_main, y_main, test_size=0.1765, random_state=42)
def copy_files(images, labels, image_dir, label_dir, dest_image_dir, dest_label_dir):
    for img, lbl in zip(images, labels):
        shutil.copy(os.path.join(image_dir, img), os.path.join(dest_image_dir, img))
        shutil.copy(os.path.join(label_dir, lbl), os.path.join(dest_label_dir, lbl))
copy_files(x_train, y_train, image_dir, label_dir, train_image_dir, train_label_dir)
copy_files(x_val, y_val, image_dir, label_dir, val_image_dir, val_label_dir)
copy_files(x_test, y_test, image_dir, label_dir, test_image_dir, test_label_dir)
print(f"Training set: {len(x_train)} images")
print(f"Validation set: {len(x_val)} images")
print(f"Test set: {len(x_test)} images")

In [ ]:
#Clone the YOLOv5 Model from its github repo

!git clone https://github.com/ultralytics/yolov5
!pip install -U -r yolov5/requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 16725, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (196/196), done.
remote: Total 16725 (delta 140), reused 152 (delta 70), pack-reused 16459
Receiving objects: 100% (16725/16725), 15.41 MiB | 29.33 MiB/s, done.
Resolving deltas: 100% (11443/11443), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.5/290.5 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 20.0 MB/s eta 0:00:00
 

In [ ]:
#Defining number of classes based on the yaml file so that it can be used on our dataset

import yaml
with open('PATH_TO_THE_FOLDER_CONTAINING_THE_DATA.YAML_FILE_OF_THE_DATASET' + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#Default yaml file of the YOLOv5 model

%cat /content/yolov5/models/yolov5s.yaml

# Ultralytics YOLOv5 🚀, AGPL-3.0 license

# Parameters
nc: 80 # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 4],

In [ ]:
#Enable the modification of the write file to be able to customise it as per requirements of your model

from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate PATH_TO_THE_YOLOV5_MODEL_WHICH_IS_TO_BE_TRAINED


nc: CUSTOM_NUMBER_OF_CLASSES
depth_multiple: 0.33
width_multiple: 0.50

anchors:
  - [10,13, 16,30, 33,23]
  - [30,61, 62,45, 59,119]
  - [116,90, 156,198, 373,326]

backbone:
  [[-1, 1, Focus, [64, 3]],
   [-1, 1, Conv, [128, 3, 2]],
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],
  ]

head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],
   [-1, 3, BottleneckCSP, [512, False]],

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],
   [-1, 3, BottleneckCSP, [256, False]],

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],
   [-1, 3, BottleneckCSP, [512, False]],

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],
   [-1, 3, BottleneckCSP, [1024, False]],

   [[17, 20, 23], 1, Detect, [nc, anchors]],
  ]

In [ ]:
#Check whether GPU is being used

import torch
print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

torch 2.3.0+cu121 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15102MB, multi_processor_count=40)


In [ ]:
#Training the model

%%time
%cd /content/yolov5/
!python train.py --img 640 --batch 16 --epochs 90 --data PATH_TO_THE_DATA.YAML_FILE_OF_THE_DATASET --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

/content/yolov5
2024-07-04 10:00:15.818483: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 10:00:15.818539: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 10:00:15.908999: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=, cfg=./models/custom_yolov5s.yaml, data=/content/drive/MyDrive/fire-hazard-detection-2/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=90, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cac

In [ ]:
#Testing the model for object detection thus detecting PPE,sparks,flammable material,HEMs and people

!pip install ultralytics
model_weights = 'PATH_TO_THE_CUSTOM_TRAINED_YOLO_MODEL'
test_data = 'PATH_TO_THE_FOLDER_CONTAINING_THE_TESTING_IMAGES'
%cd /content/drive/MyDrive/yolov5/
!python detect.py --weights {model_weights} --img 640 --conf 0.25 --source {test_data}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 9.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
#Testing for fire Hazards

import torch
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
model = torch.hub.load('ultralytics/yolov5','custom',path='PATH_TO_THE_CUSTOM_TRAINED_MODEL')
video_path = 'PATH_TO_THE_TESTING_VIDEO'
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    results = model(frame)
    sparks_coords = []
    inflammable_coords = []
    for detection in results.xyxy[0].cpu().numpy():
        x1, y1, x2, y2, confidence, class_id = detection
        center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2
        label = f'{results.names[int(class_id)]} {confidence:.2f}'
        color = (0, 100, 100) if int(class_id) == 0 else (0, 255, 0)
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        cv2.circle(frame, (int(center_x), int(center_y)), 3, color, -1)
        if int(class_id) == 0:
            inflammable_coords.append((center_x, center_y))
        elif int(class_id) == 1:
            sparks_coords.append((center_x, center_y))
    if sparks_coords and inflammable_coords:
        for sc in sparks_coords:
            for ic in inflammable_coords:
                distance = (np.sqrt((sc[0] - ic[0]) ** 2 + (sc[1] - ic[1]) ** 2))*0.002097
                midpoint_x, midpoint_y = (sc[0] + ic[0]) / 2, (sc[1] + ic[1]) / 2
                cv2.line(frame, (int(sc[0]), int(sc[1])), (int(ic[0]), int(ic[1])), (255, 255, 255), 1)
                if(distance<=1.5):
                  cv2.putText(frame, f'Inflammable material in Danger Zone: {distance:.2f}', (int(midpoint_x), int(midpoint_y) - 10),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                else:
                  cv2.putText(frame, f'Inflammable Material in Safe Zone: {distance:.2f}', (int(midpoint_x), int(midpoint_y) - 10),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    out.write(frame)
cap.release()
out.release()
cv2.destroyAllWindows()

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-7-9 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7249215 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
#Danger Zone Detection

import torch
import cv2
import numpy as np
model = torch.hub.load('ultralytics/yolov5', 'custom', path='PATH_TO_THE_TRAINED_MODEL', force_reload=True)
def euclidean_distance(box1, box2):
    center1 = ((box1[0] + box1[2]) / 2, (box1[1] + box1[3]) / 2)
    center2 = ((box2[0] + box2[2]) / 2, (box2[1] + box2[3]) / 2)
    distance = np.sqrt((center1[0] - center2[0])**2 + (center1[1] - center2[1])**2)
    return distance, center1, center2
def intersects(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1[:4]
    x2_min, y2_min, x2_max, y2_max = box2[:4]
    return not (x1_max < x2_min or x1_min > x2_max or y1_max < y2_min or y1_min > y2_max)
cap = cv2.VideoCapture('PATH_TO_INPUT_VIDEO')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('NAME_OF_OUTPUT_VIDEO', fourcc, fps, (width, height))
previous_vehicle_positions = {}
initial_distances = {}
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    results = model(frame)
    boxes = results.xyxy[0].cpu().numpy()
    labels = results.names
    current_vehicle_positions = {}
    for i, box in enumerate(boxes):
        x1, y1, x2, y2, conf, cls = box
        cls = int(cls)
        label = labels[cls]
        color = (0, 255, 0) if cls == 0 else (0, 0, 255)
        if cls == 0:
            vehicle_id = i
            current_vehicle_positions[vehicle_id] = ((x1 + x2) / 2, (y1 + y2) / 2)
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    for vehicle_id, current_position in current_vehicle_positions.items():
        if vehicle_id in previous_vehicle_positions:
            previous_position = previous_vehicle_positions[vehicle_id]
            movement_distance = np.sqrt((current_position[0] - previous_position[0])**2 + (current_position[1] - previous_position[1])**2)
            if movement_distance > 10:
                moving = True
            else:
                moving = False
            if moving:
                for j, box2 in enumerate(boxes):
                    cls2 = int(box2[5])
                    if cls2 == 1:
                        if intersects(box1, box2):
                            cv2.putText(frame, 'Alert: Intersection', (int(current_position[0]), int(current_position[1]) - 40),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                        else:
                            distance, center1, center2 = euclidean_distance(box1, box2)
                            if vehicle_id not in initial_distances:
                                initial_distances[vehicle_id] = distance
                            else:
                                if distance <= initial_distances[vehicle_id] / 2:
                                    cv2.putText(frame, 'Alert: Distance Decreased', (int(center1[0]), int(center1[1]) - 40),
                                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
    previous_vehicle_positions = current_vehicle_positions.copy()
    out.write(frame)
cap.release()
out.release()
cv2.destroyAllWindows()
